In [1]:
import gzip
from yargy import Parser, rule, or_
from yargy.pipelines import morph_pipeline, caseless_pipeline
from yargy.interpretation import fact
from yargy.predicates import gram, dictionary, type as yargy_type, eq
import re
from typing import Optional
from dataclasses import dataclass

# --- Structure des données extraites
@dataclass
class Entry:
    name: str
    birth_date: Optional[str]
    birth_place: Optional[str]

# --- Règles de reconnaissance des entités
Person = fact(
    'Person',
    ['first', 'last']
)

NAME_RULE = rule(
    gram('Name').interpretation(Person.first),
    gram('Name').interpretation(Person.last)
)

DATE_RULE = rule(
    yargy_type('INT'),  # Jour
    or_(
        morph_pipeline(['января', 'февраля', 'марта', 'апреля', 'мая', 'июня',
                    'июля', 'августа', 'сентября', 'октября', 'ноября', 'декабря']),
        rule(yargy_type('INT'))  # Cas d'une date au format 15/05/1990
    ),
    yargy_type('INT')  # Année
).interpretation(fact('Date', ['day', 'month', 'year']))

DATE_SHORT_RULE = rule(
    yargy_type('INT'),
    eq('/'),
    yargy_type('INT'),
    eq('/'),
    yargy_type('INT')
).interpretation(fact('Date', ['day', 'month', 'year']))

BIRTHPLACE_RULE = rule(
    morph_pipeline([
        'Paris', 'Marseille', 'Lyon', 'Toulouse', 'Nice', 'France', 
        'Londres', 'Berlin', 'Moscou', 'New York', 'Tokyo', 'Bruxelles', 
        'Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 
        'Казань', 'Россия'

    ])
)

BIRTH_PHRASE_RULE = rule(
    dictionary({'родился', 'родилась'}),
    eq('в'),
    BIRTHPLACE_RULE.interpretation(fact('BirthPlace', ['location'])),
    dictionary({'в'}),
    or_(DATE_RULE, DATE_SHORT_RULE).interpretation(fact('Date', ['day', 'month', 'year']))
)

parser_name = Parser(NAME_RULE)
parser_date = Parser(DATE_RULE)
parser_short_date = Parser(DATE_SHORT_RULE)
parser_birthplace = Parser(BIRTHPLACE_RULE)
parser_birth_phrase = Parser(BIRTH_PHRASE_RULE)

# --- Fonction d'extraction des informations
def extract_information(text: str) -> Entry:
    name = None
    birth_date = None
    birth_place = None

    for match in parser_name.findall(text):
        name = ' '.join([_.value for _ in match.tokens])
        break  # On prend la première correspondance

    for match in parser_date.findall(text):
        day, month, year = [_.value for _ in match.tokens]
        birth_date = f'{day} {month} {year}'
        break

    for match in parser_short_date.findall(text):
        day, month, year = [_.value for _ in match.tokens]
        birth_date = f'{day}/{month}/{year}'
        break

    for match in parser_birthplace.findall(text):
        birth_place = match.tokens[0].value
        break

    for match in parser_birth_phrase.findall(text):
        tokens = [_.value for _ in match.tokens]
        birth_place = tokens[2]
        birth_date = '/'.join(tokens[4:7])
        break

    return Entry(name=name, birth_date=birth_date, birth_place=birth_place)

# --- Lecture du fichier GZ
def process_gz_file(file_path: str):
    with gzip.open(file_path, 'rt', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                entry = extract_information(line)
                if entry.name or entry.birth_date or entry.birth_place:
                    print(entry)

# --- Lancer le traitement
process_gz_file('news.txt.gz')


Entry(name='Матс Сундин', birth_date='12 ноября 2012', birth_place=None)
Entry(name='Владимир Филиппов', birth_date=None, birth_place='Казань')
Entry(name='О том', birth_date=None, birth_place='России')
Entry(name='Киитиро Тоёды', birth_date=None, birth_place=None)
Entry(name='Бена Николсона', birth_date='24 февраля 2016', birth_place=None)
Entry(name=None, birth_date='1 июня 2015', birth_place='России')
Entry(name=None, birth_date='22 сентября 2011', birth_place=None)
Entry(name=None, birth_date='12 октября 2016', birth_place='России')
Entry(name=None, birth_date=None, birth_place='Россия')
Entry(name='Игорь Денисов', birth_date=None, birth_place='России')
Entry(name='море Из', birth_date='7 июня 2016', birth_place='России')
Entry(name='Ханс Ульрих', birth_date=None, birth_place=None)
Entry(name=None, birth_date=None, birth_place='France')
Entry(name=None, birth_date='19 июня 2014', birth_place='России')
Entry(name='Энди Коулсон', birth_date=None, birth_place='New')
Entry(name=None, b

ValueError: too many values to unpack (expected 3)

In [2]:
import gzip
from yargy import Parser, rule, or_
from yargy.pipelines import morph_pipeline, caseless_pipeline
from yargy.interpretation import fact
from yargy.predicates import gram, dictionary, type as yargy_type, eq
import re
from typing import Optional
from dataclasses import dataclass

# --- Structure des données extraites
@dataclass
class Entry:
    name: str
    birth_date: Optional[str]
    birth_place: Optional[str]

# --- Règles de reconnaissance des entités
Person = fact(
    'Person',
    ['first', 'last']
)

NAME_RULE = rule(
    gram('Name').interpretation(Person.first),
    gram('Name').interpretation(Person.last)
)

DATE_RULE = rule(
    yargy_type('INT'),  # Jour
    or_(
        morph_pipeline(['января', 'февраля', 'марта', 'апреля', 'мая', 'июня',
                    'июля', 'августа', 'сентября', 'октября', 'ноября', 'декабря']),
        rule(yargy_type('INT'))  # Cas d'une date au format 15/05/1990
    ),
    yargy_type('INT')  # Année
).interpretation(fact('Date', ['day', 'month', 'year']))

DATE_SHORT_RULE = rule(
    yargy_type('INT'),
    eq('/'),
    yargy_type('INT'),
    eq('/'),
    yargy_type('INT')
).interpretation(fact('Date', ['day', 'month', 'year']))

BIRTHPLACE_RULE = rule(
    morph_pipeline([
        'Paris', 'Marseille', 'Lyon', 'Toulouse', 'Nice', 'France', 
        'Londres', 'Berlin', 'Moscou', 'New York', 'Tokyo', 'Bruxelles', 
        'Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 
        'Казань', 'Россия'

    ])
)

BIRTH_PHRASE_RULE = rule(
    dictionary({'родился', 'родилась'}),
    eq('в'),
    BIRTHPLACE_RULE.interpretation(fact('BirthPlace', ['location'])),
    dictionary({'в'}),
    or_(DATE_RULE, DATE_SHORT_RULE).interpretation(fact('Date', ['day', 'month', 'year']))
)

parser_name = Parser(NAME_RULE)
parser_date = Parser(DATE_RULE)
parser_short_date = Parser(DATE_SHORT_RULE)
parser_birthplace = Parser(BIRTHPLACE_RULE)
parser_birth_phrase = Parser(BIRTH_PHRASE_RULE)

# --- Fonction d'extraction des informations
def extract_information(text: str) -> Entry:
    name = "Non spécifié"
    birth_date = "Non spécifié"
    birth_place = "Non spécifié"

    # Extraction du nom
    for match in parser_name.findall(text):
        name = ' '.join([_.value for _ in match.tokens])
        break

    # Extraction de la date complète
    for match in parser_date.findall(text):
        tokens = [_.value for _ in match.tokens]
        if len(tokens) >= 3:  # Vérifie qu'il y a au moins 3 tokens
            day, month, year = tokens[:3]
            birth_date = f'{day} {month} {year}'
            break

    # Extraction de la date au format court
    for match in parser_short_date.findall(text):
        tokens = [_.value for _ in match.tokens]
        if len(tokens) >= 3:  # Vérifie qu'il y a au moins 3 tokens
            day, month, year = tokens[:3]
            birth_date = f'{day}/{month}/{year}'
            break

    # Extraction du lieu
    for match in parser_birthplace.findall(text):
        birth_place = match.tokens[0].value
        break

    # Extraction phrase complète (lieu + date)
    for match in parser_birth_phrase.findall(text):
        tokens = [_.value for _ in match.tokens]
        if len(tokens) >= 7:  # Vérifie qu'il y a au moins 7 tokens pour "родился в <lieu> в <date>"
            birth_place = tokens[2]
            birth_date = '/'.join(tokens[4:7])
            break

    return Entry(name=name, birth_date=birth_date, birth_place=birth_place)

# --- Lecture du fichier GZ
def process_gz_file(file_path: str):
    with gzip.open(file_path, 'rt', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                entry = extract_information(line)
                if entry.name or entry.birth_date or entry.birth_place:
                    print(entry)

# --- Lancer le traitement
process_gz_file('news.txt.gz')

Entry(name='Non spécifié', birth_date='Non spécifié', birth_place='Non spécifié')
Entry(name='Матс Сундин', birth_date='12 ноября 2012', birth_place='Non spécifié')
Entry(name='Владимир Филиппов', birth_date='Non spécifié', birth_place='Казань')
Entry(name='Non spécifié', birth_date='Non spécifié', birth_place='Non spécifié')
Entry(name='О том', birth_date='Non spécifié', birth_place='России')
Entry(name='Киитиро Тоёды', birth_date='Non spécifié', birth_place='Non spécifié')
Entry(name='Бена Николсона', birth_date='24 февраля 2016', birth_place='Non spécifié')
Entry(name='Non spécifié', birth_date='1 июня 2015', birth_place='России')
Entry(name='Non spécifié', birth_date='22 сентября 2011', birth_place='Non spécifié')
Entry(name='Non spécifié', birth_date='12 октября 2016', birth_place='России')
Entry(name='Non spécifié', birth_date='Non spécifié', birth_place='Россия')
Entry(name='Игорь Денисов', birth_date='Non spécifié', birth_place='России')
Entry(name='Non spécifié', birth_date='N